# Análisis de la vida académica de la Facultad de Ciencias
## Perfil de Profesores

### Carga de librerías para el análisis 

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re
import matplotlib.pyplot as plt

### Cargar bases de datos que utilizaremos

In [2]:
df_courses=pd.read_csv('../Data/courses.csv', dtype = {'ID': str,'CourseName': str}).set_index('ID')

df_groups=pd.read_csv('../Data/groups.csv', dtype = {'GroupYSID': str,'Quota': int,'Signed':int}).set_index('GroupYSID')

df_profs=pd.read_csv('../Data/profs-all.csv', dtype = {'ID': int, 'Name':str, 'Appointment':str,'SimpAppointment': 'category'}).set_index('ID')

df_teaching=pd.read_csv('../Data/teaching.csv', dtype = {'ProfID': int,'GroupYSID': str,'ProfType':'category', 'Days': str, 'Hours': str})

df_plans=pd.read_csv('../Data/plans_courses.csv', dtype={'CourseID':int,'PlanID':"int64",'CourseType':'category'})

df_plan_names=pd.read_csv('../Data/plans.csv').set_index('ID')

# Hay algunas materias homólogas que no queremos que estén
df_homologas=pd.read_csv('../Data/homologas.csv')
df_homologas['old_name_code']=df_homologas['old_name_code'].apply(lambda x: '{:0>4d}'.format(x))
df_homologas['new_name_code']=df_homologas['new_name_code'].apply(lambda x: '{:0>4d}'.format(x))

In [3]:
for df in [df_courses, df_groups, df_profs, df_teaching, df_plans]:
    display(df.sample(3))

,CourseName
ID,
1813,Planeación Estratégica
0199,Introducción al Estudio de los Recursos Pesque...
0459,Educación Ambiental


,Quota,Signed
GroupYSID,,
2008-1-1700-5540,0,10
2020-1-0001-4185,61,44
2007-2-1204-5075,25,25


,Name,Mail,Appointment,SimpAppointment
ID,,,,
46045,Act. Camilo Baez Mendoza,camilo.baez@ciencias.unam.mx,Ayudante de Profesor de Asignatura,Ayud
63922,Lic. Jorge Antonio García Galicia,Escribenos,Ayudante de Profesor de Asignatura,Ayud
76043,M. en C. Esther Lara González,elara@email.ifc.unam.mx,Profesora de Asignatura,PA


,ProfID,GroupYSID,ProfType,Days,Hours
12803,29203,2015-1-0005-4114,Ayudante,NaN,NaN
113414,62131,2021-2-1323-7030,Profesor,ma ju,16 a 17:30
119112,64308,2016-1-0094-4171,Ayudante,lu mi vi,14 a 15


,CourseID,PlanID,CourseType
301,1049,106,Optativa
335,249,106,Optativa
534,408,122,Optativa


### Agregar funciones útiles

In [4]:
def get_course_name(courseid):
    try:
        if type(courseid)==int:
            courseid='{:0>4d}'.format(courseid)
        if type(courseid)==str:
            return(df_courses.loc[courseid]['CourseName'])
    except:
        return "Not found"
    
def es_asignatura(text):
    if text=='PA':
        return(1)
    else:
        return(0)

### Agregar columnas útiles

Para `df_teaching`

In [5]:
df_teaching['CourseID']=df_teaching['GroupYSID'].apply(lambda x: x.split('-')[-2])
df_teaching['Semester']=df_teaching['GroupYSID'].apply(lambda x:x[:6])
df_teaching['CourseName']=df_teaching['CourseID'].apply(get_course_name)
display(df_teaching['CourseID'].head())
display(df_teaching['Semester'].head())
display(df_teaching['CourseName'].head())

0    2017
1    0307
2    0827
3    0302
4    0302
Name: CourseID, dtype: object

0    2016-1
1    2006-2
2    2008-1
3    2007-1
4    2007-2
Name: Semester, dtype: object

0                          Meteorología
1    Máquinas Digitales con Laboratorio
2        Dinámica de Medios Deformables
3                  Fenómenos Colectivos
4                  Fenómenos Colectivos
Name: CourseName, dtype: object

Para `df_profs`

In [6]:
df_profs['Asignatura']=df_profs['SimpAppointment'].apply(es_asignatura)

### Tipos de semestre

In [7]:
sem_1=['{}-1'.format(j) for j in range(2001,2022)]
sem_2=['{}-2'.format(j) for j in range(2001,2022)]
sem_3=['{}-3'.format(j) for j in range(2001,2022)]
sem_4=['{}-4'.format(j) for j in range(2001,2022)]
sem_regular=sorted(sem_1+sem_2)
sem_inter=sorted(sem_3+sem_4)
sem_1_recent=['{}-1'.format(j) for j in range(2017,2022)]
sem_2_recent=['{}-2'.format(j) for j in range(2017,2022)]
sem_3_recent=['{}-3'.format(j) for j in range(2017,2022)]
sem_4_recent=['{}-4'.format(j) for j in range(2017,2022)]
sem_regular_recent=sorted(sem_1_recent+sem_2_recent)
sem_inter_recent=sorted(sem_3_recent+sem_3_recent)

## Join de la información

In [8]:
df_teaching_join=df_teaching.join(df_groups,on='GroupYSID').join(df_profs, on='ProfID')
df_teaching_join['Asignatura']=df_teaching_join['SimpAppointment'].apply(es_asignatura)
df_teaching_join.head()

# Quitamos las materias homólogas. Dejamos sólo con el nombre reciente.
print(df_teaching_join.shape)
df_teaching_join=df_teaching_join[~df_teaching_join['CourseID'].isin(df_homologas['old_name_code'])]
print(df_teaching_join.shape)

(146660, 15)
(140357, 15)


In [9]:
df_teaching_join.sample(5)

,ProfID,GroupYSID,ProfType,Days,Hours,CourseID,Semester,CourseName,Quota,Signed,Name,Mail,Appointment,SimpAppointment,Asignatura
144475,91802,2017-2-1603-5423,Profesor,NaN,NaN,1603,2017-2,Taller Nivel 2,0,3,Adriana Hernández Ángeles,haadriana@yahoo.com.mx,Profesora de Asignatura,PA,1.0
102963,58076,2016-1-1800-5451,Profesor,NaN,NaN,1800,2016-1,Taller Nivel 4,0,12,M. en C. Alya Ramos Ramos Elorduy,alya.ramos@ciencias.unam.mx,Profesora de Asignatura,PA,1.0
35114,33466,2006-2-0419-8044,Discusión,lu mi vi,9 a 10,0419,2006-2,Electromagnetismo I,0,22,Dr. Guillermo Manuel Herrera Pérez,guillermo.herrera@nucleares.unam.mx,Profesor de Asignatura,PA,1.0
56248,34332,2001-1-0180-2042,Ayudante,ma ju sá,10 a 13,0180,2001-1,Edafología,6,6,Biól. Claudia Vallejo Albarrán,claudiavalleal@yahoo.com.mx,Profesora de Asignatura,PA,1.0
75163,42332,2010-2-0429-7050,Ayud. Lab.,mi,16 a 18,0429,2010-2,Temas Selectos de Ingeniería de Software A,0,7,Rodrigo Jiménez del Valle,rocko@ciencias.unam.mx,Ayudante de Profesor de Asignatura,Ayud,0.0


## Profesores actuales del departamento de matemáticas

In [10]:
df_matematicas=df_plans[df_plans.PlanID==122].CourseID.apply(lambda x: '{:0>4d}'.format(x))
df_matematicas

466    0007
467    0091
468    0244
469    0249
470    0008
       ... 
738    0032
739    0034
740    0053
741    0686
742    0580
Name: CourseID, Length: 277, dtype: object

In [11]:
filtro_1=df_teaching_join['Semester']=='2021-2'
filtro_2=df_teaching_join['CourseID'].isin(df_matematicas)
filtro_3=df_teaching_join['ProfType']=='Profesor'

In [12]:
df_aux=df_teaching_join[filtro_1&filtro_2&filtro_3][['ProfID','CourseName','Name','CourseID','Quota','Signed','Appointment','SimpAppointment','Asignatura']]
df_aux

,ProfID,CourseName,Name,CourseID,Quota,Signed,Appointment,SimpAppointment,Asignatura
42,1987,Cálculo Diferencial e Integral II,Lic. Alberto Manuel Aldama Garisoain,0092,65,33,Profesor de Asignatura,PA,1.0
106,2138,Seminario Matemáticas Aplicadas I,Dr. Pedro Eduardo Miramontes Vidal,0735,65,15,Profesor de Carrera Titular,PTC,0.0
439,4102,Cálculo Diferencial e Integral I,Mat. José Santos,0091,65,10,Profesor de Asignatura,PA,1.0
754,4944,Álgebra Superior I,M. en C. Jorge Ortiz Espejel,0007,65,10,Profesor de Asignatura,PA,1.0
968,5478,Fenómenos Colectivos,Dra. Patricia Goldstein Menache,0302,60,60,Profesora de Carrera Titular,PTC,0.0
...,...,...,...,...,...,...,...,...,...
146423,101173,Álgebra Superior II,Mtra. en Matemáticas Arilín Susana Haro Palma,0008,137,137,Profesora de Asignatura,PA,1.0
146588,101508,Conjuntos y Lógica,M. en C. Sonia Navarro Flores,0271,65,12,Profesora de Asignatura,PA,1.0
146633,103773,México: Nación Multicultural,María Teresa Romero Tovar,0291,60,50,Profesora de Asignatura,PA,1.0
146652,103809,Seminario de Topología A,Dr. Carlos Segovia González,0977,65,1,Profesor de Asignatura,PA,1.0


In [13]:
df_aux2=df_aux.groupby(['ProfID']).agg({'CourseID':'count','Signed':'sum','Name':'first','Appointment':'first'}).sort_values(by='Signed',ascending=False)
df_aux2.head(20)

,CourseID,Signed,Name,Appointment
ProfID,,,,
60028,3,383,Dr. Leonardo Ignacio Martínez Sandoval,Profesor de Carrera Asociado
53980,3,356,Dra. María del Rocío Sánchez López,Profesora de Carrera Asociado
34616,1,260,Act. Javier Fernández García,Profesor de Carrera Asociado
65901,1,215,Mat. Francisco Giovanni López Sánchez,Profesor de Asignatura
59231,2,202,M. en D. Adriana León Montes,Profesora de Asignatura
33726,2,176,Dra. Ana Meda Guardiola,Profesora de Carrera Titular
72234,1,168,M. en C. Omar Patricio Juárez Álvarez,Profesor de Asignatura
58811,1,166,Dr. Alejandro Dorantes Aldama,Profesor de Asignatura
63853,2,160,Dra. Natalia Bárbara Mantilla Beniers,Profesora de Carrera Titular


In [14]:
df_aux2=df_aux[df_aux['Signed']>=100].sort_values(by='Signed',ascending=False)
df_aux2

,ProfID,CourseName,Name,CourseID,Quota,Signed,Appointment,SimpAppointment,Asignatura
61340,34616,Cálculo Diferencial e Integral II,Act. Javier Fernández García,0092,300,260,Profesor de Carrera Asociado,PTC,0.0
121297,65901,Cálculo Diferencial e Integral IV,Mat. Francisco Giovanni López Sánchez,0094,230,215,Profesor de Asignatura,PA,1.0
91413,53980,Álgebra Superior II,Dra. María del Rocío Sánchez López,0008,185,184,Profesora de Carrera Asociado,PTC,0.0
128755,72234,Cálculo Diferencial e Integral I,M. en C. Omar Patricio Juárez Álvarez,0091,175,168,Profesor de Asignatura,PA,1.0
104394,58811,Análisis Matemático I,Dr. Alejandro Dorantes Aldama,0009,170,166,Profesor de Asignatura,PA,1.0
45811,33898,Ecuaciones Diferenciales I,Dra. Laura Ortiz Bobadilla,0162,159,159,Investigadora de Carrera Titular,Inv,0.0
41690,33726,Probabilidad II,Dra. Ana Meda Guardiola,0626,150,150,Profesora de Carrera Titular,PTC,0.0
4555,18811,Cálculo Diferencial e Integral III,Mat. Mario Delgadillo Torres,0093,150,148,Profesor de Asignatura,PA,1.0
146423,101173,Álgebra Superior II,Mtra. en Matemáticas Arilín Susana Haro Palma,0008,137,137,Profesora de Asignatura,PA,1.0
108448,60028,Álgebra Superior II,Dr. Leonardo Ignacio Martínez Sandoval,0008,130,128,Profesor de Carrera Asociado,PTC,0.0


## Barchart race

In [15]:
import bar_chart_race as bcr

In [23]:
df_matematicas=df_plans[df_plans.PlanID==122].CourseID.apply(lambda x: '{:0>4d}'.format(x))
df_matematicas
filtro_1=df_teaching_join['Semester'].isin(sem_regular)
filtro_2=df_teaching_join['CourseID'].isin(df_matematicas)
filtro_3=df_teaching_join['ProfType']=='Profesor'
df_aux=df_teaching_join[filtro_1&filtro_2&filtro_3][['ProfID','CourseName','Semester','Name','CourseID','Quota','Signed','Appointment','SimpAppointment','Asignatura']]
df_aux=df_aux.groupby(['Semester','Name']).agg({'Signed':'sum'}).unstack(1).fillna(0)
df_aux.iloc[:,0:-1] = df_aux.iloc[:,0:-1].cumsum()
df_aux=df_aux['Signed']
df_aux

Name,Alfredo Castañeda Olvera,Daniel Hernández Reyes,Daniela Hérnandez Salcido,Edmundo Avendaño Mejía,Act. Abraham Ramírez Hernández,Act. Alejandra Juárez Torres,Act. Alejandro Felipe Zarate Pérez,Act. Alejandro Nava Camacho,Act. Aravid Torres Garduño,Act. Arturo Roldán López,...,Víctor Iván Moreno Oliva,Víctor Manuel Carreón Calderón,Víctor Mireles Chávez,Víctor Neumann Lara,Víctor Rufino Becerril Somera,Wilfrido Zamora Gómez,Yesenia Alfaro Ramírez,Yolanda Pimentel Alarcón,Yuriria Cortés Poza,Úrsula Jacobo Arteaga
Semester,,,,,,,,,,,,,,,,,,,,,
2001-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0
2001-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,0.0,18.0,0.0,26.0,0.0,0.0,0.0,0.0
2002-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,0.0,18.0,0.0,26.0,0.0,0.0,0.0,0.0
2002-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,0.0,18.0,0.0,38.0,0.0,0.0,0.0,0.0
2003-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,0.0,39.0,0.0,38.0,0.0,0.0,0.0,0.0
2003-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,39.0,0.0,46.0,0.0,53.0,0.0,0.0,31.0,0.0
2004-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,39.0,0.0,46.0,0.0,53.0,0.0,0.0,53.0,0.0
2004-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,39.0,0.0,46.0,0.0,65.0,0.0,0.0,64.0,0.0
2005-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,39.0,0.0,46.0,0.0,73.0,0.0,0.0,64.0,0.0


In [25]:
top_profs = set()

for index, row in df_aux.iterrows():
    top_profs |= set(row[row > 0].sort_values(ascending=False).head(10).index)

df_aux = df_aux[top_profs]

In [31]:
bcr.bar_chart_race(df = df_aux['2005-1':'2021-2'], n_bars = 10, sort='desc', title='Profesores con mas alumnos', filename = 'pl_clubs.mp4', period_length=2000,steps_per_period=30,shared_fontdict={'family': 'Helvetica'})

C:\Users\pktgc\anaconda3\lib\site-packages\bar_chart_race\_make_chart.py:286: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(self.df_values.columns)
C:\Users\pktgc\anaconda3\lib\site-packages\bar_chart_race\_make_chart.py:287: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels([max_val] * len(ax.get_xticks()))
C:\Users\pktgc\anaconda3\lib\site-packages\bar_chart_race\_make_chart.py:226: UserWarning: Some of your columns never make an appearance in the animation. To reduce color repetition, set `filter_column_colors` to `True`
  warnings.warn("Some of your columns never make an appearance in the animation. "


## Antigüedad profesores de asignatura

Un argumento que se usa frecuentemente es que "se deben dejar abiertos lugares para nuevos profesores" pues esto es algo que se hace en el Departamento de Matemáticas. En esta parte cuantificamos si esto sucede.


In [36]:
df_matematicas=df_plans[df_plans.PlanID==122].CourseID.apply(lambda x: '{:0>4d}'.format(x))
df_matematicas

466    0007
467    0091
468    0244
469    0249
470    0008
       ... 
738    0032
739    0034
740    0053
741    0686
742    0580
Name: CourseID, Length: 277, dtype: object

In [37]:
filtro_1=df_teaching_join['Semester'].isin(sem_regular)
filtro_2=df_teaching_join['CourseID'].isin(df_matematicas)
filtro_3=df_teaching_join['ProfType']=='Profesor'

df_aux=df_teaching_join[filtro_2&filtro_3][['Semester','ProfID','CourseName','Name','CourseID','Quota','Signed','Appointment','SimpAppointment','Asignatura']]
df_aux

,Semester,ProfID,CourseName,Name,CourseID,Quota,Signed,Appointment,SimpAppointment,Asignatura
25,2004-2,1987,Análisis Matemático I,Lic. Alberto Manuel Aldama Garisoain,0009,0,33,Profesor de Asignatura,PA,1.0
26,2005-1,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,53,Profesor de Asignatura,PA,1.0
27,2007-1,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,43,Profesor de Asignatura,PA,1.0
28,2010-2,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,31,Profesor de Asignatura,PA,1.0
29,2012-2,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,45,Profesor de Asignatura,PA,1.0
...,...,...,...,...,...,...,...,...,...,...
146587,2021-1,101508,Análisis Matemático II,M. en C. Sonia Navarro Flores,0010,65,7,Profesora de Asignatura,PA,1.0
146588,2021-2,101508,Conjuntos y Lógica,M. en C. Sonia Navarro Flores,0271,65,12,Profesora de Asignatura,PA,1.0
146633,2021-2,103773,México: Nación Multicultural,María Teresa Romero Tovar,0291,60,50,Profesora de Asignatura,PA,1.0
146652,2021-2,103809,Seminario de Topología A,Dr. Carlos Segovia González,0977,65,1,Profesor de Asignatura,PA,1.0


In [38]:
df_first_sem=df_aux.groupby(['ProfID']).agg({'Semester':'min'})
df_first_sem.head(10)

,Semester
ProfID,
1987,2004-2
2138,2002-1
2691,2016-2
4102,2001-1
4338,2020-1
4944,2001-1
5227,2001-1
5472,2001-1
5478,2001-1


In [39]:
df_aux2=pd.merge(df_aux,df_first_sem,on='ProfID',how='left',suffixes=('', '_new'))
df_aux2

,Semester,ProfID,CourseName,Name,CourseID,Quota,Signed,Appointment,SimpAppointment,Asignatura,Semester_new
0,2004-2,1987,Análisis Matemático I,Lic. Alberto Manuel Aldama Garisoain,0009,0,33,Profesor de Asignatura,PA,1.0,2004-2
1,2005-1,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,53,Profesor de Asignatura,PA,1.0,2004-2
2,2007-1,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,43,Profesor de Asignatura,PA,1.0,2004-2
3,2010-2,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,31,Profesor de Asignatura,PA,1.0,2004-2
4,2012-2,1987,Cálculo Diferencial e Integral I,Lic. Alberto Manuel Aldama Garisoain,0091,0,45,Profesor de Asignatura,PA,1.0,2004-2
...,...,...,...,...,...,...,...,...,...,...,...
19576,2021-1,101508,Análisis Matemático II,M. en C. Sonia Navarro Flores,0010,65,7,Profesora de Asignatura,PA,1.0,2021-1
19577,2021-2,101508,Conjuntos y Lógica,M. en C. Sonia Navarro Flores,0271,65,12,Profesora de Asignatura,PA,1.0,2021-1
19578,2021-2,103773,México: Nación Multicultural,María Teresa Romero Tovar,0291,60,50,Profesora de Asignatura,PA,1.0,2021-2
19579,2021-2,103809,Seminario de Topología A,Dr. Carlos Segovia González,0977,65,1,Profesor de Asignatura,PA,1.0,2021-2


In [41]:
df_aux3=df_aux2[df_aux2.Semester==df_aux2.Semester_new].groupby(['CourseName','Semester']).agg({'Name':'count'})
df_aux3=df_aux3.unstack(1).fillna(0).stack()
df_aux3.to_csv('vista_nuevos.csv',encoding='utf-8')

In [45]:
df_aux4=df_aux3.reset_index()
df_aux4=df_aux4[df_aux4.Semester.isin(sem_regular_recent)].groupby('CourseName').agg({'Name':sum})
df_aux4

,Name
CourseName,
Análisis Matemático I,7.0
Análisis Matemático II,2.0
Análisis Matemático III,1.0
Análisis Matemático IV,0.0
Análisis Multivariado,0.0
...,...
Álgebra Moderna III,0.0
Álgebra Moderna IV,1.0
Álgebra Superior I,13.0


In [48]:
filtro_1=df_teaching_join['Semester'].isin(sem_regular_recent)
filtro_2=df_teaching_join['CourseID'].isin(df_matematicas)
filtro_3=df_teaching_join['ProfType']=='Profesor'

df_aux5=df_teaching_join[filtro_1&filtro_2&filtro_3].groupby(['CourseName']).agg({'Name':'count'})
df_aux5

,Name
CourseName,
Análisis Matemático I,114
Análisis Matemático II,58
Análisis Matemático III,24
Análisis Matemático IV,11
Análisis Multivariado,19
...,...
Álgebra Moderna III,27
Álgebra Moderna IV,17
Álgebra Superior I,182


In [56]:
df_aux6=df_aux4.join(df_aux5,lsuffix='1').fillna(0)
df_aux6.to_csv('all_vs_new_mate.csv',encoding='utf-8')

In [ ]:
|